# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234160494301                   -0.50    7.0
  2   -7.250179314451       -1.80       -1.41    1.0
  3   -7.251309862476       -2.95       -2.05    4.0
  4   -7.251310648708       -6.10       -2.31    4.0
  5   -7.251335946440       -4.60       -2.81    4.0
  6   -7.251338403654       -5.61       -3.37    1.0
  7   -7.251338756359       -6.45       -3.84    2.0
  8   -7.251338795696       -7.41       -4.35    2.0
  9   -7.251338798367       -8.57       -4.70    2.0
 10   -7.251338798611       -9.61       -5.03    2.0
 11   -7.251338798668      -10.24       -5.25    3.0
 12   -7.251338798702      -10.47       -5.82    2.0
 13   -7.251338798704      -11.67       -6.49    3.0
 14   -7.251338798705      -12.81       -6.92    3.0
 15   -7.251338798705      -14.05       -7.28    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.062436754982387974
[ Info: Arnoldi iteration step 2: normres = 0.620239727401989
[ Info: Arnoldi iteration step 3: normres = 0.6964597099406643
[ Info: Arnoldi iteration step 4: normres = 0.2811706506566628
[ Info: Arnoldi iteration step 5: normres = 0.36923149300845187
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.29e-02, 6.02e-02, 2.61e-01, 2.52e-01, 2.73e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4150038472803924
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.31e-03, 8.41e-02, 3.81e-01, 1.08e-01, 6.50e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07444249919708958
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.67e-04, 1.02e-02, 2.46e-02, 3.11e-02, 5.56e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09575203548527253
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.87e-